# Part 1

In [1]:
filename = "inputs/12-19.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re
from functools import cache
from math import ceil

In [3]:
blueprints = {}
for l in data.splitlines():
    bp, *values = (int(v) for v in re.findall("\d+", l))
    blueprints[bp] = tuple(values)

In [4]:
@cache
def geodes(t, robots, stock, blueprint):
    ore_r, clay_r, obs_r, geo_r = robots
    ore, clay, obs, geo = stock
    (ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs) = blueprint
    # stop if we can't possibly build another geode before t = 1 (excluded)
    if geo_obs > obs + (obs_r + (clay + (clay_r + (ore + ore_r * (t - 4)) // clay_ore) * (t - 3)) // obs_clay) * (t - 2):
        return geo + t * geo_r
    else:
        # build a geode robot now if possible, as delaying geode robots is never profitable
        if ore >= geo_ore and obs >= geo_obs:
            return geodes(
                t - 1,
                (ore_r, clay_r, obs_r, geo_r + 1),
                (
                    ore + ore_r - geo_ore,
                    clay + clay_r,
                    obs + obs_r - geo_obs,
                    geo + geo_r
                ),
                blueprint
            )
        geos = [geo + t * geo_r]
        # build a geode robot
        if obs_r > 0:
            dt = max((ceil((geo_obs - obs) / obs_r), ceil((geo_ore - ore) / ore_r), 0)) + 1
            if t - dt >= 1:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r, obs_r, geo_r + 1),
                    (
                        ore + dt * ore_r - geo_ore,
                        clay + dt * clay_r,
                        obs + dt * obs_r - geo_obs,
                        geo + dt * geo_r
                    ),
                    blueprint
                ))
        # build an obsidian robot (up to a maximum)
        if clay_r > 0 and obs_r < geo_obs:
            dt = max((ceil((obs_clay - clay) / clay_r), ceil((obs_ore - ore) / ore_r), 0)) + 1
            if t - dt >= 2:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r, obs_r + 1, geo_r),
                    (
                        ore + dt * ore_r - obs_ore,
                        clay + dt * clay_r - obs_clay,
                        obs + dt * obs_r,
                        geo + dt * geo_r
                    ),
                    blueprint
                ))
        # build a clay robot (up to a maximum)
        if clay_r < obs_clay:
            dt = max(ceil((clay_ore - ore) / ore_r), 0) + 1
            if t - dt >= 3:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r + 1, obs_r, geo_r),
                    (
                        ore + dt * ore_r - clay_ore,
                        clay + dt * clay_r,
                        obs + dt * obs_r,
                        geo + dt * geo_r
                    ),
                    blueprint
                ))
        # build an ore robot (up to a maximum)
        if ore_r < max([clay_ore, obs_ore, geo_ore]):
            dt = max(ceil((ore_ore - ore) / ore_r), 0) + 1
            if t - dt >= 2:
                geos.append(geodes(
                    t - dt,
                    (ore_r + 1, clay_r, obs_r, geo_r),
                    (
                        ore + dt * ore_r - ore_ore,
                        clay + dt * clay_r,
                        obs + dt * obs_r,
                        geo + dt * geo_r
                    ),
                    blueprint
                ))

        return max(geos)

In [5]:
%%time
s = 0
for i, bp in blueprints.items():
    geo = geodes(24, (1, 0, 0, 0), (0, 0, 0, 0), bp)
    s += i * geo
s

CPU times: total: 359 ms
Wall time: 351 ms


1092

# Part 2

In [6]:
%%time
p = 1
for i, bp in list(blueprints.items())[:3]:
    geo = geodes(32, (1, 0, 0, 0), (0, 0, 0, 0), bp)
    p *= geo
p

CPU times: total: 3.38 s
Wall time: 3.38 s


3542